### TEM Beta Lactamase Analysis 001

This is the general work file which will contain all the analysis of the beta lactamase.

In [3]:
%reload_ext autoreload
%autoreload 2
import os
import sys
from pyeed.core import ProteinInfo, Alignment
from pyeed.aligners import PairwiseAligner

##### starting point for analysis

The starting point for the beta lactamase will be a TEM Protein. Possibly TEM 01 but this is not certain.

In [4]:
starting_protein_tem = ProteinInfo.get_id("QGC48744.1")

##### simple search & save & aligment

Here we search with blast in NCBI database and create the aligments.

In [5]:
n_hits = 10
e_value = 0.05

In [8]:
blast_results_starting_protein = starting_protein_tem.ncbi_blastp(n_hits=n_hits, e_value=e_value)

🏃🏼‍♀️ Running PBLAST
╭── protein name: TEM family beta-lactamase
├── accession: QGC48744.1
├── organism: Escherichia coli
├── e-value: 0.05
╰── max hits: 10


In [ ]:
# save the blast search results
current_path = os.path.dirname(os.getcwd())
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "data_blast_search_{}_{}_{}".format(starting_protein_tem.source_id.replace('.', ''), n_hits, int(e_value*1000)))
os.makedirs(output_folder_blast_search, exist_ok=True)
for hit in blast_results_starting_protein:
    with open(output_folder_blast_search + "/{}.json".format(hit.id), "w") as f:
        f.write(hit.json())

NameError: name 'blast_results_starting_protein' is not defined

In [ ]:
# create pairwise aligments
mode = "global"
alignment_blast_search = Alignment.from_sequences(blast_results_starting_protein, aligner=PairwiseAligner, mode=mode)
# save the pairwise alignments
output_folder_alignments = os.path.join(current_path, "TEM-lactamase", "data", "data_alignments_pairwise_{}_{}_{}_{}".format(mode, starting_protein_tem.source_id.replace('.', ''), n_hits, int(e_value*1000)))
os.makedirs(output_folder_alignments, exist_ok=True)
for alignment in alignment_blast_search:
    with open(output_folder_alignments + "/{}.json".format(alignment.id), "w") as f:
        f.write(alignment.json())

NameError: name 'blast_results_starting_protein' is not defined

##### network building & visualize

In [ ]:
%reload_ext autoreload
%autoreload 2
# read in the saved aligments results
n_hits = 10
e_value = 0.05
mode = 'global'
name = starting_protein_tem.source_id.replace('.', '')
read_alignment_folder = os.path.join(current_path, "TEM-lactamase", "data", "data_alignments_pairwise_{}_{}_{}_{}".format(mode, name, n_hits, int(e_value*1000)))
alignment_list = []
for file in os.listdir(read_alignment_folder):
    with open(os.path.join(read_alignment_folder, file), "r") as f:
        alignment_list.append(Alignment.from_json(f.read()))

ValidationError: 1 validation error for Alignment
input_sequences
  Invalid sequence type. Sequences must be of type AbstractSequence or Sequence (type=value_error)